# Solve sudokus automatically

![0](https://miro.medium.com/max/445/1*4fqzGBs9HY9hmbZSZmIPtw.jpeg)

Explanation of sudoku terminology. Source: https://www.pinterest.com/pin/468796642443225725/

First things first. If you are not familiar with the rules of a sudoku puzzle, don’t be frightened. They are quite simple:
You must fill the boxes of a 9x9 grid with numbers from 1 to 9 in a way that there are no recurring numbers in each row, column, and 3x3 block.
If you take a look at the image, the only number that fits into the top left box is two since it’s the only one that doesn’t appear in this row, column and box.

REF: Puzzle 8: You Won't Want to Play Sudoku Again

by MIT OpenCourseWare: https://www.youtube.com/watch?v=auK3PSZoidc

### Data Structure
To solve this programmatically for a 9x9 grid, I needed a data structure for the original, unsolved Grid and another grid for Possible Values. Each grid was a python 2-D list. The data structure to hold the grid and list of possibles took me awhile to decide upon. In the end, a 2D array of 9x9 lists worked best for me, and I calculated the necessary indexes when the “cubes”, aka 3x3 submatrix or mini-grids, needed to be inspected.

### Example sudoku and data structures

Let’s take a random sudoku puzzle from the Internet and try to solve it with Python. I propose that we handle this puzzle as a two-dimensional Python array where empty boxes are represented with zeros and other boxes with corresponding numbers. We should obtain the following:

In [25]:
sudoku = [[8, 1, 0, 0, 3, 0, 0, 2, 7],
          [0, 6, 2, 0, 5, 0, 0, 9, 0], 
          [0, 7, 0, 0, 0, 0, 0, 0, 0], 
          [0, 9, 0, 6, 0, 0, 1, 0, 0], 
          [1, 0, 0, 0, 2, 0, 0, 0, 4], 
          [0, 0, 8, 0, 0, 5, 0, 7, 0], 
          [0, 0, 0, 0, 0, 0, 0, 8, 0], 
          [0, 2, 0, 0, 1, 0, 7, 5, 0], 
          [3, 8, 0, 0, 7, 0, 0, 4, 2]]

### Making it look a lot more like a real sudoku

Sure, this two-dimensional python array does not look like real sudoku without any gridlines. Hence, it would be great to print out the current state of the puzzle. Below is the code example along with its output.

In [26]:
def printsudoku():
    print("\n")
    for i in range(len(sudoku)):
        line = ""
        if i == 3 or i == 6:
            print("---------------------")
        for j in range(len(sudoku[i])):
            if j == 3 or j == 6:
                line += "| "
            line += str(sudoku[i][j])+" "
        print(line)

In [27]:
printsudoku()



8 1 0 | 0 3 0 | 0 2 7 
0 6 2 | 0 5 0 | 0 9 0 
0 7 0 | 0 0 0 | 0 0 0 
---------------------
0 9 0 | 6 0 0 | 1 0 0 
1 0 0 | 0 2 0 | 0 0 4 
0 0 8 | 0 0 5 | 0 7 0 
---------------------
0 0 0 | 0 0 0 | 0 8 0 
0 2 0 | 0 1 0 | 7 5 0 
3 8 0 | 0 7 0 | 0 4 2 


#### Step 1. Finding the unfilled cells
To solve a certain cell, we must first find the row and column number of a cell that’s empty. The following function does the trick.

In [28]:
def findNextCellToFill(sudoku):
    for x in range(9):
        for y in range(9):
            if sudoku[x][y] == 0:
                return x, y
    return -1, -1

This iterates through all the columns in the first row, then second row and so on. Whenever it encounters an empty cell, the function returns the corresponding indexes. Simple!

#### Step 2. Validating entries

Suppose we have an entry e. We need a function that checks whether it violates the three main rules of sudoku when placed to the i-th row and j-th column. The following function returns True if none of the three rules are violated. Otherwise, the function will return False.

In [29]:
def isValid(sudoku, i, j, e):
    rowOk = all([e != sudoku[i][x] for x in range(9)])
    if rowOk:
        columnOk = all([e != sudoku[x][j] for x in range(9)])
        if columnOk:
            secTopX, secTopY = 3*(i//3), 3*(j//3)
            for x in range(secTopX, secTopX+3):
                for y in range(secTopY, secTopY+3):
                    if sudoku[x][y] == e:
                        return False
            return True
    return False

Three rules are examined:
1. rowOk: this one checks if there are no repeating numbers in the i-th row;
2. columnOk: this one makes sure if there are no repeating numbers in the j-th column;
3. If both rowOk and columnOk are True then the lines 6–10 check if the entry fits a certain block.

#### Step 3. Solving the puzzle

Now it’s all about solving the given sudoku. We’ll do it with the following function:

In [30]:
def solveSudoku(sudoku, i=0, j=0):
    i, j = findNextCellToFill(sudoku)
    if i == -1:
        return True
    for e in range(1, 10):
        if isValid(sudoku, i, j, e):
            sudoku[i][j] = e
            if solveSudoku(sudoku, i, j):
                return True
            sudoku[i][j] = 0
    return False

The first three lines of the function make sure that we have an empty cell in our puzzle. If i is equal to -1 then we have completed solving our puzzle.

If there is an empty cell at i-th row and j-th column then it will try to fit all the possible entries from 1 to 9 into this box. If the entry is valid (verified by the isValid function) then tries to solve the sudoku recursively under the assumption that this is a correct entry.

If our assumption was incorrect then the 9-th line would return False and that sets the value of a cell in i-th row and j-th column to zero.

In [31]:
solveSudoku(sudoku)

True

In [32]:
printsudoku()



8 1 9 | 4 3 6 | 5 2 7 
4 6 2 | 7 5 1 | 3 9 8 
5 7 3 | 2 9 8 | 4 1 6 
---------------------
2 9 4 | 6 8 7 | 1 3 5 
1 5 7 | 9 2 3 | 8 6 4 
6 3 8 | 1 4 5 | 2 7 9 
---------------------
7 4 5 | 3 6 2 | 9 8 1 
9 2 6 | 8 1 4 | 7 5 3 
3 8 1 | 5 7 9 | 6 4 2 


### Algorithm

I needed to track the active Grid and a grid of Possible Values. At first I was going to process the Grid iteratively, but then it became apparent that only the Possible Values grid need to be updated. If the Possible Values resulted in one number, then the cell is solved. Assign that value to the active Grid and eliminate that value from the other Possible Values in the corresponding Row, Column, and “Cube” (Please refer to the highlighted areas in the graphic below). My definition of a “cube” is a 3x3 submatrix within the 9x9 original grid. Nine “cubes” fit in the grid.

Below is how I envisioned my solution.

![0](https://miro.medium.com/max/722/1*JDyDPpvNaUdZ_5fXoluf9w.png)

The Possible Values are in small font. In the graphic, the cell being inspected is (3, 5) or the 4th row, 6th column and is highlighted in purple. If that inspected cell value is determined to be “7”, then a “7” cannot be in any cell in Row 3, nor Column 5, nor the middle “Cube.” Keep searching for Possible Values where the count is 1, update the Grid and Possible Values, until the puzzle is solved.


To generate the Possible Values, I used python’s set() feature. For each of the 81 cells, I would start with full set of 1–9 numbers and remove() those numbers that were not logically possible using set() subtraction.
If a working Grid’s cell has the correct value, then there are no Possible Values, and thus assigned a 0. Otherwise, each cell is assigned the full set of numbers from 1 to 9 minus the “conflicts” in each Row, Cell, and Cube by using set() subtraction in turn.

Iteration vs Recursion for the Solution

After I programmatically solved sudoku, a colleague reminded me of the N-Queens backtracking algorithm, where recursion can be used. He also pointed out that the worst case complexity is O(81!).
To me, this seems like a brute force method to try all possibilities. Since there are many relationships between numbers, they should be analyzed before “platonic” guesses should be attempted. This reminds me (for lack of a better name), my “value in a cube row/column” trick was not implemented. If a value is known in 2 out of 3 in a contiguous, cubic row/column (3x1 or 1x3 submatrix), the value has to be in the remaining row/column, respectively). This trick would solve a few more sudoku puzzles, and I’m sure other other tricks exist as well. I am not a sudoku expert, but I suppose the N-Queens method could be used as a last-resort rather than as the #1 choice as a hybrid implementation. Or, platonic iteration could be tried further as well.
In general, I still prefer iteration over recursion because the latter takes too much stack and memory resources. Although recursion is useful in some applications, and code can look cleaner, it can be harder to debug if the base case is set incorrectly.